In [2]:
%%bash
mv /Users/aadib/Desktop/Screen\ Shot\ 2020-06-15\ at\ 1.02.50\ PM.png ./img/spark-storage.png
![alt text](./img/spark-1.png)

![alt text](./img/spark-storage.png)

#### Let's compare the primitive RDD with Modern DataFrames

In [ ]:
from pyspark import SparkContext, SparkConf

conf = SparkConf()
conf.set('spark.driver.memory', '48gb')
conf.set('spark.executor.memory','10gb') 
conf.set('spark.executor.cores', '5') 
conf.set('spark.executor.instances', '30')

sc = SparkContext(conf=conf)